<a href="https://colab.research.google.com/github/kramerkraus/2155-CP3-mkraus/blob/main/diffusionmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
%reset -f

In [27]:
#free up all ram
import gc
gc.collect()

4346

In [ ]:
#make it so i can access files in the folder 2155-CP3-mkraus
import sys
sys.path.append('/content/2155-CP3-mkraus/')
from utils import *
from evaluate import *
import random

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Load dataset from CSV files
data_dir = '/content/2155-CP3-mkraus/dataset'
splits = load_dataset_splits(data_dir)

# Get feature names from the CSV file
feature_names = pd.read_csv(os.path.join(data_dir, 'train_original.csv')).columns.tolist()
print(f"\n✓ Features loaded: {len(feature_names)} features")
print(f"Feature names: {feature_names[:5]}...{feature_names[-5:]}")  # Show first and last 5

In [ ]:
# Data exploration and analysis
print("\n" + "="*70)
print("DATASET ANALYSIS")
print("="*70)

# Extract data for easier access
X_train = splits['train']['imputed']
mask_train = splits['train']['missing_mask']
X_train_original = splits['train']['original']


# for i in [18,14, 5]:
#   print(i)
#   X_traincpy = X_train.copy()

#   #pick a random number between 0 and 36
#   # k = random.randint(0,36)

#   X_traincpy[:,i] = -1
#   #append X_traincpy to the bottom of X_train
#   X_train = np.vstack((X_train, X_traincpy))

#   mask_traincpy = mask_train.copy()
#   mask_traincpy[:,i] = 1

#   #append mask_traincpy to the bottom of mask_train
#   mask_train = np.vstack((mask_train, mask_traincpy))

#   #append a copy of X_train_original to the bottom of itself
#   X_train_original = np.vstack((X_train_original, X_train_original))

for i in range(37):
  print(i)
  X_traincpy = X_train.copy()
  X_traincpy = X_traincpy[0:300,:]

  #pick a random number between 0 and 36
  # k = random.randint(0,36)

  X_traincpy[:,i] = -1
  #append X_traincpy to the bottom of X_train
  X_train = np.vstack((X_train, X_traincpy))

  mask_traincpy = mask_train.copy()
  mask_traincpy = mask_traincpy[0:300,:]
  mask_traincpy[:,i] = 1

  #append mask_traincpy to the bottom of mask_train
  mask_train = np.vstack((mask_train, mask_traincpy))

  #append a copy of X_train_original to the bottom of itself
  X_train_original = np.vstack((X_train_original, X_train_original[0:300,:]))




X_val = splits['val']['imputed']
mask_val = splits['val']['missing_mask']
X_val_original = splits['val']['original']

X_test = splits['test']['imputed']
mask_test = splits['test']['missing_mask']
X_test_original = splits['test']['original']

# Test2 data (no original available for evaluation)
X_test2 = splits['test2']['imputed']
mask_test2 = splits['test2']['missing_mask']

print(f"\nData shapes:")
print(f"  - Training: {X_train.shape}")
print(f"  - Validation: {X_val.shape}")
print(f"  - Test: {X_test.shape}")
print(f"  - Test2: {X_test2.shape} (evaluation set - no ground truth)")

In [ ]:
# Data Preprocessing (Handle Missing Values)

print("\n" + "="*70)
print("DATA PREPROCESSING")
print("="*70)

# Handle missing values properly
print("Processing missing values and preparing data...")
print("Mask convention: True=missing, False=observed (in original masks)")

print(f"\n✓ Data preprocessing completed successfully")
print(f"  - Training data range: [{X_train_original[~mask_train].min():.3f}, {X_train_original[~mask_train].max():.3f}]")
print(f"  - Validation data range: [{X_val_original[~mask_val].min():.3f}, {X_val_original[~mask_val].max():.3f}]")
print(f"  - Test data range: [{X_test_original[~mask_test].min():.3f}, {X_test_original[~mask_test].max():.3f}]")

# Create data loaders
batch_size = 64
print(f"\nCreating data loaders with batch size: {batch_size}")

train_dataset = TensorDataset(torch.FloatTensor(X_train_original), torch.FloatTensor((~mask_train).astype(float)))
val_dataset = TensorDataset(torch.FloatTensor(X_val_original), torch.FloatTensor((~mask_val).astype(float)))
test_dataset = TensorDataset(torch.FloatTensor(X_test_original), torch.FloatTensor((~mask_test).astype(float)))
test2_dataset = TensorDataset(torch.FloatTensor(X_test2), torch.FloatTensor((~mask_test2).astype(float)))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
test2_loader = DataLoader(test2_dataset, batch_size=batch_size, shuffle=False)

# Preview a batch
sample_batch_data, sample_batch_mask = next(iter(train_loader))
print(f"\nSample batch shape: {sample_batch_data.shape}")
print(f"Sample batch mask shape: {sample_batch_mask.shape}")
print(f"Sample batch missing percentage: {(sample_batch_mask == 0).float().mean().item()*100:.1f}%")  # 0 = missing in model tensors


In [ ]:
class TransformerDenoiser(nn.Module):
    def __init__(self, input_dim=37, d_model=4096, n_heads=64, depth=24):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model

        # Project the raw vector into a transformer-hidden dimension
        self.input_proj = nn.Linear(input_dim, d_model)

        # Mask embedding (tells transformer which elements were observed)
        self.mask_emb = nn.Linear(input_dim, d_model)

        # Time embedding
        self.time_emb = nn.Sequential(
            nn.Linear(1, d_model),
            nn.SiLU(),
            nn.Linear(d_model, d_model),
        )

        # Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=4*d_model,
            batch_first=True,
            activation="gelu"
        )

        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=depth
        )

        # Project back to noise dimension
        self.out_proj = nn.Linear(d_model, input_dim)

    def forward(self, x_t, t, mask):
        """
        x_t:  [B, D]
        t:    [B, 1]
        mask: [B, D]
        """
        B, D = x_t.shape

        # project inputs
        h = self.input_proj(x_t)                    # [B, d]
        m = self.mask_emb(mask)                    # [B, d]
        te = self.time_emb(t)                      # [B, d]

        h = h + m + te                              # fuse conditioning

        # Transformer expects sequence; treat entire vector as 1 token
        h = h.unsqueeze(1)                          # [B, 1, d]

        h = self.transformer(h)                     # [B, 1, d]

        out = self.out_proj(h.squeeze(1))           # [B, D]
        return out


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


# --------------------------------------------------
# 1. Small MLP backbone (denoiser)
# --------------------------------------------------

class MLPDenoiser(nn.Module):
    def __init__(self, input_dim=37, hidden=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim + 1 + input_dim, hidden), # x + t + mask
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, input_dim),
        )

    def forward(self, x_t, t, mask):
        """
        x_t: noised data
        t: time step (batch, 1)
        mask: binary mask (1 = observed, 0 = missing)
        """
        t = t / 1000.0       # normalize time
        inp = torch.cat([x_t, t, mask], dim=1)
        return self.net(inp)


# --------------------------------------------------
# 2. Diffusion core (betas, schedules, noise)
# --------------------------------------------------

def make_beta_schedule(T=1000, start=1e-4, end=0.03):
    return torch.linspace(start, end, T)


class Diffusion(nn.Module):
    def __init__(self, input_dim, timesteps=1000):
        super().__init__()
        self.T = timesteps
        betas = make_beta_schedule(timesteps)
        alphas = 1.0 - betas
        alphas_cum = torch.cumprod(alphas, dim=0)

        self.register_buffer("betas", betas)
        self.register_buffer("alphas", alphas)
        self.register_buffer("alphas_cum", alphas_cum)

        #self.model = MLPDenoiser(input_dim=input_dim)
        self.model = TransformerDenoiser(input_dim=input_dim)

    # -------------------------
    # q(x_t | x_0)
    # -------------------------
    def q_sample(self, x0, t, noise=None):
        if noise is None:
            noise = torch.randn_like(x0)
        a_bar = self.alphas_cum[t].unsqueeze(1)
        return torch.sqrt(a_bar) * x0 + torch.sqrt(1 - a_bar) * noise, noise

    # -------------------------
    # Training step
    # -------------------------
    def forward(self, x0, mask):
        """
        x0 : clean data (batch, D)
        mask : 1 = observed, 0 = missing
        """
        B = x0.shape[0]
        device = x0.device

        # Random time t for each sample
        t = torch.randint(0, self.T, (B,), device=device)

        # Noise forward
        xt, noise = self.q_sample(x0, t, noise=None)

        # Condition on observed values
        xt = xt * (1 - mask) + x0 * mask

        # Predict noise
        noise_pred = self.model(xt, t.unsqueeze(1).float(), mask)

        # Loss only on missing entries
        loss = ((noise_pred - noise) ** 2 * (1 - mask)).mean()
        return loss

    # -------------------------
    # Sampling / imputation
    # -------------------------
    @torch.no_grad()
    def sample(self, x_obs, mask, guidance_scale=2.0):
      """x_obs has missing entries set to anything (will overwrite them).
      guidance_scale: w in cfg (>=1). w=1.0 -> no guidance, w>1.0 -> stronger conditioning.
      """

      x = torch.randn_like(x_obs)

      # Define the target categories for Feature 26
      feature_26_idx = 25 # Assuming 'Feature 26' is at index 25 (0-indexed)
      allowed_categories = torch.tensor([0.0, 0.5, 1.0], device=x.device, dtype=torch.float32)

      feature_25_idx = 24

      for t in reversed(range(self.T)):
          bt = self.betas[t]
          at = self.alphas[t]
          a_bar = self.alphas_cum[t]

          # ----- prepare two inputs: uncond and cond -----
          # For conditioned input, always respect observed values
          x_cond = x * (1 - mask) + x_obs * mask

          # For unconditioned input, do not inject observed values (i.e., no overwrite)
          x_uncond = x  # same x but without overwritten observed entries

          # t tensor
          t_tensor = torch.full((x.shape[0], 1), t, device=x.device).float()

          # Predict noise for uncond and cond
          noise_uncond = self.model(x_uncond, t_tensor, torch.zeros_like(mask))
          noise_cond = self.model(x_cond, t_tensor, mask)

          # Combine with guidance
          noise_pred = noise_uncond + guidance_scale * (noise_cond - noise_uncond)

          # DDPM update step (use combined noise_pred)
          coef1 = 1 / torch.sqrt(at)
          coef2 = (1 - at) / torch.sqrt(1 - a_bar)

          x = coef1 * (x - coef2 * noise_pred)

          if t > 0:
              x += torch.sqrt(bt) * torch.randn_like(x)

          # After update, ensure we keep observed values exactly (optional safety)
          x = x * (1 - mask) + x_obs * mask

          # --- Guidance for Feature 26 ---
          # Find samples where Feature 26 was originally missing
          missing_for_f26 = (mask[:, feature_26_idx] == 0)
          if missing_for_f26.any():
              # Get the current imputed values for Feature 26 for these samples
              values_to_constrain = x[missing_for_f26, feature_26_idx]

              # Calculate absolute differences to each allowed category
              diffs = torch.abs(values_to_constrain.unsqueeze(1) - allowed_categories)

              # Find the index of the closest allowed category
              closest_category_indices = torch.argmin(diffs, dim=1)

              # Snap the imputed values to the closest allowed category
              x[missing_for_f26, feature_26_idx] = allowed_categories[closest_category_indices]



           # --- Guidance for Feature 25 ---
          # Find samples where Feature 26 was originally missing
          missing_for_f25 = (mask[:, feature_25_idx] == 0)
          if missing_for_f25.any():
              # Get the current imputed values for Feature 25 for these samples
              values_to_constrain = x[missing_for_f25, feature_25_idx]

              # Calculate absolute differences to each allowed category
              diffs = torch.abs(values_to_constrain.unsqueeze(1) - allowed_categories)

              # Find the index of the closest allowed category
              closest_category_indices = torch.argmin(diffs, dim=1)

              # Snap the imputed values to the closest allowed category
              x[missing_for_f25, feature_25_idx] = allowed_categories[closest_category_indices]




      # xmin = x.min(dim=0, keepdim=True).values
      # xmax = x.max(dim=0, keepdim=True).values

      # x_scaled = (x - xmin) / (xmax - xmin + 1e-12)


      return x # x_scaled

In [25]:
model = Diffusion(input_dim=37, timesteps=18).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-4)

In [26]:
for epoch in range(200):
    for batch, mask in train_loader:
        batch = batch.to(device)
        mask = mask.to(device)

        loss = model(batch, mask)

        opt.zero_grad()
        loss.backward()
        opt.step()

    print(f"epoch {epoch} | loss {loss.item():.4f}")

AcceleratorError: CUDA error: out of memory
Search for `cudaErrorMemoryAllocation' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# x_incomplete = X_test_imputed[i]      # has -1 replaced by something
# mask = (X_test_missing_mask[i] == 0)  # convert your mask to 1/0 observed/missing

# x_incomplete = torch.tensor(x_incomplete).float().to(device)
# mask = torch.tensor(mask).float().to(device)

# samples = []

# for _ in range(20):
#     x_gen = model.sample(x_incomplete.unsqueeze(0), mask.unsqueeze(0))
#     samples.append(x_gen.cpu().numpy())

# samples = np.array(samples)   # (20, 1, 37)

In [ ]:
def evaluate_imputation(model, data_loader, device):
    """Evaluate imputation performance."""
    model.eval()

    all_imputations = []
    all_originals = []
    all_masks = []

    with torch.no_grad():
        for batch_data_original, batch_mask_original in data_loader:
            batch_data_original = batch_data_original.to(device)
            batch_mask_original = batch_mask_original.to(device)

            # Use the model's sampling method for imputation
            imputed_data = model.sample(batch_data_original, batch_mask_original)

            all_imputations.append(imputed_data.cpu().numpy())
            all_originals.append(batch_data_original.cpu().numpy())
            all_masks.append(batch_mask_original.cpu().numpy())

    # Concatenate all results
    imputations = np.vstack(all_imputations)
    originals = np.vstack(all_originals)
    masks = np.vstack(all_masks)

    return imputations, originals, masks

In [ ]:
print("\n" + "="*70)
print("MODEL EVALUATION")
print("="*70)

# Evaluate on test set
print("Evaluating model on test set...")
test_imputations, test_originals, test_masks = evaluate_imputation(
    model, test_loader, device
)

print(f"✓ Test set evaluation completed")
print(f"  - Test samples: {test_imputations.shape[0]}")
print(f"  - Features: {test_imputations.shape[1]}")

test_imputations_denorm = test_imputations  # Already in original scale
test_original_denorm = X_test_original  # Already in original scale

# Calculate comprehensive metrics
print("\nCalculating comprehensive metrics...")
feature_metrics = {}

# Create masks for missing values (where we need to evaluate imputation)
missing_mask = (test_masks == 0)  # True where values were missing (0 in model tensors = missing)

for i, feature_name in enumerate(feature_names):
    if missing_mask[:, i].sum() > 0:  # Only evaluate features with missing values
        # Get imputed and ground truth values for missing positions only
        imputed_missing = test_imputations_denorm[missing_mask[:, i], i]
        ground_truth_missing = test_original_denorm[missing_mask[:, i], i]

        # Calculate metrics
        mse = mean_squared_error(ground_truth_missing, imputed_missing)
        mae = mean_absolute_error(ground_truth_missing, imputed_missing)

        # Correlation
        try:
            correlation = np.corrcoef(ground_truth_missing, imputed_missing)[0, 1]
        except:
            correlation = np.nan

        # Mean difference and Jensen-Shannon divergence
        mean_diff, js_div = calculate_jsd_and_mean_diff(
            imputed_missing, ground_truth_missing, feature_name
        )

        feature_metrics[feature_name] = {
            'n_missing': missing_mask[:, i].sum(),
            'mse': mse,
            'mae': mae,
            'correlation': correlation,
            'mean_difference': mean_diff,
            'js_divergence': js_div,
        }

print(f"✓ Metrics calculated for {len(feature_metrics)} features with missing values")

# Display metrics for last 4 features (as requested)
print(f"\n" + "="*100)
print("METRICS FOR LAST 4 FEATURES")
print("="*100)
print(f"{'Feature':<15} {'N_Miss':<8} {'MSE':<10} {'MAE':<10} {'Corr':<8} {'Mean_Diff':<10} {'JS_Div':<8}")
print("-" * 100)

last_4_features = list(feature_metrics.keys())[-4:] if len(feature_metrics) >= 4 else list(feature_metrics.keys())

for feature in last_4_features:
    metrics = feature_metrics[feature]
    print(f"{feature:<15} {metrics['n_missing']:<8} {metrics['mse']:<10.4f} {metrics['mae']:<10.4f} "
          f"{metrics['correlation']:<8.3f} {metrics['mean_difference']:<10.4f} {metrics['js_divergence']:<8.4f} ")

# Summary statistics
all_mse = [m['mse'] for m in feature_metrics.values() if not np.isnan(m['mse'])]
all_mae = [m['mae'] for m in feature_metrics.values() if not np.isnan(m['mae'])]
all_corr = [m['correlation'] for m in feature_metrics.values() if not np.isnan(m['correlation'])]
all_mean_diff = [m['mean_difference'] for m in feature_metrics.values() if not np.isnan(m['mean_difference'])]
all_js_div = [m['js_divergence'] for m in feature_metrics.values() if not np.isnan(m['js_divergence'])]

print(f"\nSummary Statistics Across All Features:")
print(f"  - Average MSE: {np.mean(all_mse):.4f} ± {np.std(all_mse):.4f}")
print(f"  - Average MAE: {np.mean(all_mae):.4f} ± {np.std(all_mae):.4f}")
print(f"  - Average Correlation: {np.mean(all_corr):.3f} ± {np.std(all_corr):.3f}")
print(f"  - Average Mean Difference: {np.mean(all_mean_diff):.4f} ± {np.std(all_mean_diff):.4f}")
print(f"  - Average JS Divergence: {np.mean(all_js_div):.4f} ± {np.std(all_js_div):.4f}")

In [ ]:
# Create the visualization
plot_prediction_scatter(test_imputations_denorm, test_original_denorm, test_masks, feature_names, n_features=37)



In [ ]:
# Distribution comparison plots
plot_distribution_comparison(test_imputations_denorm, test_original_denorm,
                             test_masks, feature_names, n_features=25)


In [ ]:
def generate_samples(model, X_test, test_loader, device, n_samples_per_test=100):
    """Generate multiple samples for a dataset using the trained model.
    """
    # We'll generate multiple samples
    test_samples = np.zeros((X_test.shape[0], n_samples_per_test, X_test.shape[1]))

    # Set model to evaluation mode
    model.eval()

    with torch.no_grad():
        # Create a progress bar for all samples
        from tqdm import tqdm

        for batch_idx, (batch_data, batch_mask) in enumerate(tqdm(test_loader, desc="Generating Samples")):
            batch_data = batch_data.to(device)
            batch_mask = batch_mask.to(device)

            # Calculate the indices for this batch
            start_idx = batch_idx * test_loader.batch_size
            end_idx = min(start_idx + test_loader.batch_size, X_test.shape[0])
            actual_batch_size = end_idx - start_idx

            # Generate multiple samples for each item in the batch
            for j in range(n_samples_per_test):
                # Get imputed sample using the model's sampling method
                imputed = model.sample(batch_data, batch_mask)

                # Store the samples (already in original scale since we didn't normalize)
                test_samples[start_idx:end_idx, j, :] = imputed.cpu().numpy()
    print(f"✓ Generated samples shape: {test_samples.shape}")
    print(f"  - {test_samples.shape[0]} samples")
    print(f"  - {test_samples.shape[1]} generated variations per sample")
    print(f"  - {test_samples.shape[2]} features per sample")

    # Data is already in original scale (no denormalization needed)
    test_samples_final = test_samples.copy()

    # Calculate summary statistics
    mean_across_samples = test_samples_final.mean(axis=1)  # Mean across the 100 samples

    print(f"  - Range of means: [{mean_across_samples.min():.4f}, {mean_across_samples.max():.4f}]")

    return test_samples

In [ ]:
# Test Evaluation

print("="*70)
print("TEST EVALUATION")
print("="*70)

# Generate multiple samples for test using the trained model
print(f"Generating 100 samples for each of {X_test.shape[0]} test samples...")

test_samples = generate_samples(
    model, X_test, test_loader, device, n_samples_per_test=100
)

In [ ]:
from evaluate import *
test_score = compute_score(generated_samples=test_samples, set_name='test')
print("Test score:", test_score)

In [ ]:
# Test2 Evaluation

print("="*70)
print("TEST2 EVALUATION")
print("="*70)

# Generate multiple samples for test2 using the trained model
print(f"Generating 100 samples for each of {X_test2.shape[0]} test2 samples...")

test2_samples = generate_samples(
    model, X_test2, test2_loader, device, n_samples_per_test=100
)

In [ ]:
rng = np.random.default_rng()
id = rng.integers(1e8, 1e9-1)
np.save(f"{id}.npy", test2_samples)